In [8]:
!pip install tensorflow
!pip install keras

     |████████████████████████████████| 173.9 MB 2.1 kB/s eta 0:00:01
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 3.3 MB 3.4 MB/s eta 0:00:01
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     |████████████████████████████████| 10.6 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 3.1 MB/s eta 0:00:01
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
     |████████████████████████████████| 781 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 2.5 MB/s eta 0:00:01
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached wrapt-1.12.1-cp38-

In [7]:
import matplotlib.pyplot as plt
import pandas as pd

import os
import yaml

from functools import partial
from tqdm import tqdm
from keras.layers import (Activation, Convolution1D, Convolution2D, Dense,
                          Dropout, Embedding, Flatten, Merge, Input)
from keras.layers.merge import Concatenate, concatenate
from keras.regularizers import l2
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
import keras.backend as K

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [ ]:
def conv1D_distribution(n_components,
                        n_bins,
                        week_embedding_size):
    """
    One dimensional conv model over input distribution to give an output
    distribution

    Merges two branches
    - predictions : (batch_size, n_bins, n_models)
    - weeks : (batch_size, week_embedding_size)

    Parameters
    ----------
    n_components : int
        Number of component distributions
    n_bins : int
        Number of bins in the prediction distribution
    week_embedding_size : int
        Embedding vector size for week
    """

    dist_input = Input(shape=(n_bins, n_components))
    week_input = Input(shape=(week_embedding_size,))

    # Dist branch
    if n_bins < 50:
        dist_x = Convolution1D(32, 5, padding="same")(dist_input)
        dist_x = Convolution1D(10, 3, padding="same")(dist_x)
    else:
        dist_x = Convolution1D(10, 3, padding="same")(dist_input)
        dist_x = Convolution1D(10, 3, padding="same")(dist_x)

    dist_x = Flatten()(dist_x)
    dist_x = Dense(10, activation="tanh")(dist_x)

    week_x = Dense(5, activation="tanh")(week_input)

    merged = concatenate([dist_x, week_x])
    merged = Dense(20, activation="relu")(merged)
    merged = Dense(n_bins, activation="softmax")(merged)
    
    model = Model(inputs=[dist_input, week_input], outputs=merged)

    return model